# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,55.99,94,0,4.47,RU,1689969334
1,1,tiksi,71.6872,128.8694,45.37,67,22,7.90,RU,1689969334
2,2,edinburgh of the seven seas,-37.0676,-12.3116,53.91,61,100,16.93,SH,1689969334
3,3,bethel,41.3712,-73.4140,84.88,67,20,12.66,US,1689969288
4,4,rio grande,-32.0350,-52.0986,77.11,87,8,14.29,BR,1689969335


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    hover_cols=["City", "Country", "Max Temperature", "Current Description"],
    tiles="OSM",
    color="City",
    size="Humidity",
    alpha=0.7,
    frame_width=800,
    frame_height=600,
    xlabel="Longitude",
    ylabel="Latitude",
)


# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Max Temp < 85 but > 68, Wind Speed < 5, and 0 cloudiness and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 85)
    & (city_data_df["Max Temp"] > 65)
    & (city_data_df["Wind Speed"] < 5)
    & (city_data_df["Cloudiness"] <10)
].dropna()

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,college,64.8569,-147.8028,74.46,64,0,0.00,US,1689969343
70,70,porto uniao,-26.2381,-51.0783,66.25,80,0,3.36,BR,1689969170
108,108,port douglas,-16.4833,145.4667,68.74,80,1,4.16,AU,1689969365
144,144,jincheng,35.5022,112.8328,69.37,86,0,2.51,CN,1689969384
184,184,lander,42.8330,-108.7307,82.80,31,0,4.61,US,1689969370
203,203,buka,40.8108,69.1986,76.14,27,0,1.19,UZ,1689969403
222,222,kendari,-3.9450,122.4989,72.41,95,2,2.68,ID,1689969409
284,284,azadshahr,34.7909,48.5707,77.49,21,0,4.61,IR,1689969428
306,306,ribeira brava,32.6500,-17.0667,75.70,74,7,2.89,PT,1689969440
406,406,urzhar,47.0930,81.6294,72.72,36,2,4.85,KZ,1689969471


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Reset the index
hotel_df.reset_index(drop=True, inplace=True)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,college,US,64.8569,-147.8028,64,
1,porto uniao,BR,-26.2381,-51.0783,80,
2,port douglas,AU,-16.4833,145.4667,80,
3,jincheng,CN,35.5022,112.8328,86,
4,lander,US,42.8330,-108.7307,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
porto uniao - nearest hotel: Hotel Flórida
port douglas - nearest hotel: Mantra Aqueous Port Douglas
jincheng - nearest hotel: 德豪宾馆
lander - nearest hotel: Rodeway Inn Pronghorn Lodge
buka - nearest hotel: No hotel found
kendari - nearest hotel: City Hotel Kendari
azadshahr - nearest hotel: هتل آموزش
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
urzhar - nearest hotel: Sharikan
old town - nearest hotel: Terasa U Prince
gorele - nearest hotel: No hotel found
birao - nearest hotel: No hotel found
toktogul - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,college,US,64.8569,-147.8028,64,BEST WESTERN PLUS Pioneer Park Inn
1,porto uniao,BR,-26.2381,-51.0783,80,Hotel Flórida
2,port douglas,AU,-16.4833,145.4667,80,Mantra Aqueous Port Douglas
3,jincheng,CN,35.5022,112.8328,86,德豪宾馆
4,lander,US,42.8330,-108.7307,31,Rodeway Inn Pronghorn Lodge
5,buka,UZ,40.8108,69.1986,27,No hotel found
6,kendari,ID,-3.9450,122.4989,95,City Hotel Kendari
7,azadshahr,IR,34.7909,48.5707,21,هتل آموزش
8,ribeira brava,PT,32.6500,-17.0667,74,Cheerfulway Bravamar Hotel
9,urzhar,KZ,47.0930,81.6294,36,Sharikan


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Drop rows with no hotel found
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    hover_cols=["Hotel Name", "City", "Country"],
    tiles="OSM",
    color="Hotel Name",
    size="Humidity",
    alpha=0.7,
    frame_width=800,
    frame_height=600,
    xlabel="Longitude",
    ylabel="Latitude",
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)